# Week 9 - Macro Facility Basics (Part 3)

* Creating macro variables using PROC SQL

[Jupyter Python Notebook Keyboard Shortcuts and Text Snippets for Beginners](http://maxmelnick.com/2016/04/19/python-beginner-tips-and-tricks.html)

#### Create a macro variable using PROC SQL INTO and then pass it into a DATA step 
##### The PROC SQL code below uses the SELECT and INTO statements to store the value of average weight [ave(weight)] in a macro variable. 

In [7]:
*Ex9_macro_vars_transfer.sas (Part 1); 
options nodate nonumber ps=58;
proc sql noprint;
 select avg(weight) format=6.2 
    into :m_avg_wt 
    from sashelp.class;
 quit; 
%put &m_avg_wt ;

In [37]:
*Ex11_one_multiple_mvars_sql.sas (Part 1);
ods html close;
options nonotes nosource nodate nonumber;
proc sql noprint;
 select count(*)
        INTO :nobs
FROM SASHELP.CARS;
quit;
%put Number of Observations = %SYSFUNC(LEFT(&nobs));
run;

In [15]:
options nodate nonumber nosource;
ods html close;
options nonotes nosource nodate nonumber;
proc sql noprint;
 select distinct make
        INTO :make_list separated by ','
        FROM SASHELP.CARS;
quit;
%put &=make_list;

In [30]:
proc sql;
select memname, name from dictionary.columns
where libname= upcase("sashelp") and
name like "%Status";
quit;


Member Name,Column Name
BMT,Status
BURROWS,Status
HEART,Status
HEART,Chol_Status
HEART,BP_Status
HEART,Weight_Status
HEART,Smoking_Status


In [16]:
proc sql noprint ;
select distinct quote(trim(make)) into :make_q_list separated by ','
from sashelp.cars;
quit ;
%put &=make_q_list;

In [25]:

proc sql noprint ;
select distinct "'"||left(trim(make))||"'" into :make_sq_list
separated by ','
from sashelp.cars;
quit ;
%put &=make_sq_list;

       

### Note for the code in the next cell
#### Notice that the macro variables follow an INTO clause and are preceded by a colon. Here the PROC SQL creates a series of macro variables, each with its own distinct value (often referred to as vertical list of macro variables of the form &MAKES1, and so on. 

#### When PROC SQL is executed, a series of macro variables including &SQLOBS is generated and placed in the most local symbol table.  The value of the macro variable &SQLOBS is the number of rows processed by the SELECT statement.

#### When you submit this macro, it is compiled and then stored in the default catalog work.sasmacr.


In [40]:
*Ex11_one_multiple_mvars_sql.sas (Part 3);
%let Put_title = List of Values into a Series of Macro Variables;
proc sql noprint;
 select distinct make
        INTO :makes1-
  FROM SASHELP.CARS ;
 %put Number of Rows: &sqlobs;
quit;
%macro reveal;
 %put &Put_title;
 %Do i=1 %TO &Sqlobs;
    %put &&makes&i;
  %end;
%mend reveal;
%reveal


[A Hands-on Introduction to SAS® Metadata DICTIONARY Tables
and SASHELP Views](https://www.mwsug.org/proceedings/2018/HW/MWSUG-2018-HW-9.pdf)

In [1]:
* Ex29_symputx_sql_into.sas (Part 1);
options nocenter nodate nonumber;
* Adapted from Simon (2017);
proc means data=sashelp.class noprint;
 var weight;
 output out=mystats mean=avweight;
 run;

data _null_;
 set mystats;
 call symputx('meanweight',avweight);
 run;

proc print data=sashelp.class noobs;
var name sex weight;
 where weight > &meanweight;
title 'Students weighing more than average weight';
title2 "Average weight:(%sysfunc(round(&meanweight, 0.01)) lbs)";
title3 '3 DATA/PROC Steps';
run;
title;

SAS Connection established. Subprocess id is 6344



Name,Sex,Weight
Alfred,M,112.5
Carol,F,102.5
Henry,M,102.5
Janet,F,112.5
Mary,F,112.0
Philip,M,150.0
Robert,M,128.0
Ronald,M,133.0
William,M,112.0


In [2]:
* Ex29_symputx_sql_into.sas (Part 2);
title 'Students weighing more than average weight';
title2 'PROC SQL - Just 1 Step';
proc sql ;
 select mean(weight) into: meanweight_x 
  FROM sashelp.class ;
 select name, sex, weight format=6.1
  FROM sashelp.class 
 having weight > &meanweight_x;
quit;